In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("distilgpt2").to("cuda")
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

accum_steps = 8
input_texts = ["The future of AI is bright."] * 4

for step in range(accum_steps):
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss / accum_steps
    loss.backward()
    print(f"Accum Step {step+1} | Loss: {loss.item():.4f}")

optimizer.step()


Accum Step 1 | Loss: 0.6129
Accum Step 2 | Loss: 0.6013
Accum Step 3 | Loss: 0.6107
Accum Step 4 | Loss: 0.6119
Accum Step 5 | Loss: 0.5961
Accum Step 6 | Loss: 0.5941
Accum Step 7 | Loss: 0.6070
Accum Step 8 | Loss: 0.5954
